In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.1.1


In [2]:
import pickle
import pandas as pd

In [3]:
with open('../3 - orchestration/model-2021-08-15.pkl', 'rb') as f_in:
    lr = pickle.load(f_in)
    
with open('../3 - orchestration/dv-2021-08-15.pkl', 'rb') as f_in:
    dv = pickle.load(f_in)

In [4]:
lr

LinearRegression()

In [11]:

def read_data(filename):
    categorical = ['PUlocationID', 'DOlocationID']
    target = 'duration'

    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df
    

In [6]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')


dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [7]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(df[target], y_pred, squared=False)
mse

11.065806009874759

In [13]:

def read_model(filename):
    with open(filename, 'rb') as f_in:
        lr = pickle.load(f_in)
    return lr
    
def read_dict_vec(filename):
    with open(filename, 'rb') as f_in:
        dv = pickle.load(f_in)
    return dv

def make_predictions(df, lr, dv):
    # preproces features
    dicts = df[categorical].to_dict(orient='records')
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)
    
    df['predictions'] = y_pred
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df_result = df[['ride_id', 'predictions']]
    return df_result

    
def predict2file(year=2021, month=2, output_file='predictions.parquet'):
    df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{year}-{month:02}.parquet')
    lr = read_model('../3 - orchestration/model-2021-08-15.pkl', 'rb')
    dv = read_dict_vec('../3 - orchestration/model-2021-08-15.pkl', 'rb')
    df_result = make_predictions(df)
    
    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )

In [9]:
make_predictions(df)

19.7 Mb